# ParFlow Short Course: Initial Conditions, Boundary Conditions, and Subsurface Properties
## Exercise: Subsurface Properties

### Domain Description
We will be working with a simple 2D box cross section domain with the following characteristics:
 - A simple cross section box domain
 - The domain is consists of a single row of 20 cells (each of which are 100 m long and 2 m wide) with 10 vertical layers of varying thickness that extend to a thickness of 48m
 - The domain slopes from right to left with a constant slope of 0.1
 - The subsurface is homogeneous
 - Simulations run for 100 hours with a time step of 1 hour.
 - There are no-flow boundary conditions on all sides except the upper x boundary (i.e. the right boundary) which has a Direchlet (constant head) boundary condition set to 1m below the surface and the top surface which has an overland flow boundary condition 
 - A constant recharge flux of 0.01m/h is applied across the top of the domain 
 - The initial head for the entire domain is set to 10m below the land surface

## Topic : Subsurface Properties

We will explore different ways of defining variable subsurface properties. The following notebook cells set up a ParFlow run by importing a ParFlow run definition, run ParFlow for 100 timesteps, and visualize the resulting ParFlow run grid. Please see [box_domain_setup_full.ipynb](https://github.com/hydroframe/parflow_short_course_updated/blob/main/initial_conditions/box_domain_setup_full.ipynb) if you would like to see the full set of keys used to define the domain for this ParFlow run. In this section we are focusing on the ParFlow keys related to [Geometries](https://parflow.readthedocs.io/en/latest/keys.html#geometries). We will explain these keys in more detail in the cells below. Also feel free to explore the [ParFlow manual](https://parflow.readthedocs.io/en/latest/keys.html#) for detail on all ParFlow keys.

We want to create a low permeability unit in the center of the domain that extends across all layers in the center three columns of the domain. We will create this using two different methods: first using a box geometry and then using an indicator file. 

In [ ]:
# Import the ParFlow package
from parflow import Run
import os
import sys
import numpy as np
from parflow.tools.fs import mkdir, cp, chdir, get_absolute_path, rm, exists
from parflow.tools.io import write_pfb

# Import functions for plotting
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../plots")))
from plots import plot_domain, plot_subsurface_storage, plot_water_balance

Method 1: Create a low permeability unit in the center of the domain using a box geometry.

In [ ]:
# Import run information from pfidb
domain_example = Run.from_definition("domain_example.pfidb")

#-----------------------------------------------------------------------------
# The Names of the GeomInputs
#-----------------------------------------------------------------------------
domain_example.GeomInput.Names = 'domain_input box2_input'

#-----------------------------------------------------------------------------
# Domain Geometry Input
#-----------------------------------------------------------------------------
domain_example.GeomInput.domain_input.InputType = 'Box'
domain_example.GeomInput.domain_input.GeomName  = 'domain'

#-----------------------------------------------------------------------------
# Domain Geometry
#-----------------------------------------------------------------------------
domain_example.Geom.domain.Lower.X = 0.0
domain_example.Geom.domain.Lower.Y = 0.0
domain_example.Geom.domain.Lower.Z = 0.0

domain_example.Geom.domain.Upper.X = 2000.0
domain_example.Geom.domain.Upper.Y = 2.0
domain_example.Geom.domain.Upper.Z = 10.0

domain_example.Geom.domain.Patches = 'x_lower x_upper y_lower y_upper z_lower z_upper'

#-----------------------------------------------------------------------------
# K2 Geometry Input
#-----------------------------------------------------------------------------
domain_example.GeomInput.box2_input.InputType = 'Box'
domain_example.GeomInput.box2_input.GeomName  = 'box2'

#-----------------------------------------------------------------------------
# K2 Geometry
#-----------------------------------------------------------------------------
domain_example.Geom.box2.Lower.X = 500.0
domain_example.Geom.box2.Lower.Y = 0.0
domain_example.Geom.box2.Lower.Z = 4.0

domain_example.Geom.box2.Upper.X = 1000.0
domain_example.Geom.box2.Upper.Y = 2.0
domain_example.Geom.box2.Upper.Z = 6.0

#-----------------------------------------------------------------------------
# Porosity
#-----------------------------------------------------------------------------
domain_example.Geom.Porosity.GeomNames      = 'domain box2'

domain_example.Geom.domain.Porosity.Type  = 'Constant'
domain_example.Geom.domain.Porosity.Value = 0.25

domain_example.Geom.box2.Porosity.Type  = 'Constant'
domain_example.Geom.box2.Porosity.Value = 0.05

#---------------------------------------------------------
# Mannings coefficient
#---------------------------------------------------------
domain_example.Geom.Mannings.GeomNames     = 'domain box2'

domain_example.Geom.domain.Mannings.Type               = 'Constant'
domain_example.Geom.domain.Mannings.GeomNames          = 'domain'
domain_example.Geom.domain.Mannings.Geom.domain.Value  = 2.e-6

domain_example.box2.Mannings.Type               = 'Constant'
domain_example.box2.Mannings.GeomNames          = 'domain'
domain_example.box2.Mannings.Geom.domain.Value  = 2.e-2

#-----------------------------------------------------------------------------
# Run ParFlow
#-----------------------------------------------------------------------------
base = os.path.join(os.getcwd(), "output")
mkdir(base)
print(f"base: {base}")
domain_example.run(working_directory=base)

Method 2: Create a low permeability unit in the center of the domain using an indicator file.

In [ ]:
# Write an indicator file from a Numpy array based on the defined computational grid
indicator = np.zeros([domain_example.ComputationalGrid.NZ,domain_example.ComputationalGrid.NY,domain_example.ComputationalGrid.NX])
print(np.shape(indicator))
indicator[4:6, 0, 4:9] = 1

write_pfb('indicator_file.pfb', indicator,dist=True)

In [ ]:
# Import run information from pfidb
domain_example = Run.from_definition("domain_example.pfidb")

#-----------------------------------------------------------------------------
# The Names of the GeomInputs
#-----------------------------------------------------------------------------
domain_example.GeomInput.Names = 'domain_input indi_input'

#-----------------------------------------------------------------------------
# Domain Geometry Input
#-----------------------------------------------------------------------------
domain_example.GeomInput.domain_input.InputType = 'Box'
domain_example.GeomInput.domain_input.GeomName  = 'domain'

#-----------------------------------------------------------------------------
# Indicator Geometry Input
#-----------------------------------------------------------------------------
domain_example.GeomInput.indi_input.InputType =   "IndicatorField"
domain_example.GeomInput.indi_input.GeomNames = "K1 K2"
domain_example.Geom.indi_input.FileName = "indicator_file.pfb"  # Created above

domain_example.GeomInput.K1.Value =    0
domain_example.GeomInput.K2.Value =    1

#-----------------------------------------------------------------------------
# Domain Geometry
#-----------------------------------------------------------------------------
domain_example.Geom.domain.Lower.X = 0.0
domain_example.Geom.domain.Lower.Y = 0.0
domain_example.Geom.domain.Lower.Z = 0.0

domain_example.Geom.domain.Upper.X = 2000.0
domain_example.Geom.domain.Upper.Y = 2.0
domain_example.Geom.domain.Upper.Z = 10.0

domain_example.Geom.domain.Patches = 'x_lower x_upper y_lower y_upper z_lower z_upper'

#-----------------------------------------------------------------------------
# Run ParFlow
#-----------------------------------------------------------------------------
base = os.path.join(os.getcwd(), "output")
mkdir(base)
print(f"base: {base}")
domain_example.run(working_directory=base)

In [ ]:
# Call plotting function for saturation and pressure
plot_domain(base, "satur", timestep=0)
plot_domain(base, "press", timestep=0)

In [ ]:
# Plot subsurface storage over time and water balance
plot_subsurface_storage(base)
plot_water_balance(base, run_name="domain_example")

In [ ]:
# Plot a map of the K field, porosity field, and indicator file

### Module Activities
After running the above code and reviewing the plots, please complete the following:
1. Using the box geometry method, change the low K unit in the domain to be a horizontal unit extending across all of layer 3 of the model. How does this affect your water balance?
1. Repeat the above exercise using the indicator file method.
1. Move the low K layer deeper in your domain. What happens? (hint: this is done by shifting the z location of the low permeability unit)